In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [10]:
total_gold_table = pd.DataFrame()
total_exp_table = pd.DataFrame()

In [4]:
driver = webdriver.Chrome('/Users/김호준/Documents/chrome_webdriver/chromedriver')
driver.implicitly_wait(3)
driver.get('http://www.op.gg/summoner/userName=ktmata')

In [5]:
driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()

In [20]:
# while True:
#     game_more_button = driver.find_elements_by_css_selector('div.GameMoreButton.Box')
#     if game_more_button == []:
#         break
#     game_more_button[0].click()
#     time.sleep(5)

In [14]:
for i in range(0, 20):
    game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail')
    game_detail_button[i].click()
    time.sleep(2)
    html = driver.page_source
    PlayerData = BeautifulSoup(html, 'html.parser')
    team_raw = PlayerData.find_all('tbody', {'class':'Content'})

    champion_list = []
    name_list = []

    for line in team_raw:
        team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
        team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})

        team_champion = []
        for champ in team_champion_raw:
            champ = champ.text.strip()
            champ = champ[:champ.find('\n')]
            team_champion.append(champ)
        champion_list.append(team_champion)

        team_name = []
        for name in team_name_raw:
            name = name.text.strip()
            team_name.append(name)
        name_list.append(team_name)
    tab_headers = driver.find_elements_by_css_selector('li.tabHeader')[5:9] #0 : 종합
                                                                        #1 : 팀 분석
                                                                        #2 : 빌드
                                                                        #3 : etc
    tab_headers[3].click()
    time.sleep(3)
    etc_gold = driver.find_element_by_css_selector('div.Content.tabItem.MatchDetailContent-gold')
    etc_html = etc_gold.get_attribute('innerHTML')
    script = BeautifulSoup(etc_html, 'html.parser').script
    graph_data = []
    str_script = str(script)
    offset = 0
    
    for idx in range(0, 20):
        start = str_script.find('"data":[', offset) + len('"data":[')
        end = str_script.find(']', start)

        graph_data.append(str_script[start:end].split(','))

        offset = end + len(']')
        
    table = pd.DataFrame(graph_data).add_suffix("분")
    gold_table = table[0:10]
    exp_table = table[10:]
    
    gold_table.insert(0, "챔피언", champion_list[0] + champion_list[1])
    gold_table.insert(0, "소환사명", name_list[0] + name_list[1])
    exp_table.insert(0, "챔피언", champion_list[0] + champion_list[1])
    exp_table.insert(0, "소환사명", name_list[0] + name_list[1])
    
    if len(gold_table.columns) > len(total_gold_table.columns):
        gold_table_columns = gold_table.columns
        exp_table_columns = exp_table.columns
    else:
        gold_table_columns = total_gold_table.columns
        exp_table_columns = total_exp_table.columns


    total_gold_table = total_gold_table.append(gold_table)[gold_table_columns]
    total_exp_table = total_exp_table.append(exp_table)[exp_table_columns]
    game_detail_button[i].click()

IndexError: list index out of range

In [13]:
total_gold_table.to_csv("total_gold_table.csv")
total_exp_table.to_csv("total_exp_table.csv")